## airtable_tools_v2

In [64]:
import os, logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from pyairtable import Table
from dotenv import load_dotenv
from datetime import datetime
import pytz

In [65]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S',level='INFO')
logger=logging.getLogger(__name__)
logger.setLevel('INFO')

In [66]:
load_dotenv()
api_key=os.getenv('AIRTABLE_API_KEY')
base_id='appDTdyxcShkSR3oF'

## Logs

In [67]:
dst_table_name='[A]Logs'

In [68]:
logger.info('Connecting to Log and Habits table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-15 19:25:53] p1018 {<ipython-input-68-0324345d546c>:1} INFO - Connecting to Log and Habits table...
[05-15 19:25:53] p1018 {<ipython-input-68-0324345d546c>:5} INFO - Connecting to [A]Logs table...


In [ ]:
logger.info('Finding Log records with "Log" in the Habit name...')
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Habit' not in o['fields']: continue
    if len(o['fields']['Habit'])>0:
        habit=habit_table.get(o['fields']['Habit'][0])
        if 'Log' in habit['fields']['Habit']:
            res.append({'Date':o['fields']['Date'], 'Score':o['fields']['Score'], 'Log': habit['fields']['Habit'].strip('Log: ')})

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
for l in list(df['Log'].unique()):
    dfs=df[df['Log']==l]
    dfs=dfs.sort_index()
    dfr=dfs.resample('D').mean() #Resample
    
    res=[]
    for d in dfr.index:
        if np.isnan(dfr.loc[d]['Score']): continue
        res.append({'Date': str(d.date()),
                 'Score': int(dfr.loc[d]['Score']),
                 'Metric': l})
    res=a.batch_create(res)
logger.info('Completed uploading %i records. ', len(res))


[05-15 19:40:50] p1018 {<ipython-input-87-afc5769bc7f9>:1} INFO - Finding Log records with "Log" in the Habit name...
  0%|          | 0/100 [00:00<?, ?it/s]

In [84]:
res

[{'Date': '2022-05-07', 'Score': 5, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-08', 'Score': 5, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-09', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-10', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-11', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-12', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-13', 'Score': 2, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-14', 'Score': 1, 'Metric': 'Satisfaction'}]

```
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "title": "[A]Logs",
  "width": "container",
  "height": "container",
  "mark": "area",
  "encoding": {
    "x": {
      "field": "Date",
      "type": "nominal"
    },
    "y": {
      "aggregate": "sum",
      "field": "Score",
      "type": "quantitative"
    },
    "color": {
      "type":"nominal",
      "field": "Metric",
      "scale": {"scheme": "category10"}
    }
  }
}
```

In [ ]:
# logger.info('Connecting to %s table...', src_table_name)
# a=Airtable(base_id, src_table_name, api_key)

# res=a.get_all()
# logger.info('Found %i records', len(res))
# res=[{**r['fields'], **r} for r in res]

# logger.info('Converting to dataframe and resampling...')
# df=pd.DataFrame(res)
# df.index=pd.DatetimeIndex(df['Date'])
# df=df.sort_index()
# dfr=df.resample('D').mean() #Resample

# logger.info('Connecting to %s table...', dst_table_name)
# a=Airtable(base_id, dst_table_name, api_key)

# logger.info('Deleting existing %s data..', dst_table_name)
# r=a.get_all()
# ids=[o['id'] for o in r]
# res=a.batch_delete(ids)

# logger.info('Uploading new analytics data...')
# l=[]
# for s in measures:
#     for d in dfr.index:
#         if np.isnan(dfr.loc[d][s]): continue
#         l.append({'Date': str(d.date()),
#                  'Measure': dfr.loc[d][s],
#                  'Metric': s})
# res=a.batch_insert(l)
# logger.info('Completed uploading %i records. ', len(res))

---

In [43]:
res=action_table.all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)

[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:2} INFO - Found 4 records
[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:5} INFO - Converting to dataframe and resampling...


In [44]:
df

,Date,Habit,Score,Goal 2,id,createdTime,fields
0,2022-05-07,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recMHleWhG91jn7sc,2022-05-08T19:00:41.000Z,"{'Date': '2022-05-07', 'Habit': ['recFqsNfUNIe..."
1,2022-05-08,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recWwwlzwY1XqGQdE,2022-05-08T21:56:53.000Z,"{'Date': '2022-05-08', 'Habit': ['recFqsNfUNIe..."
2,2022-05-08,[rec24NGEkbvktkD9B],4,[recenm1UYUwAIcoya],recmIsNssDDyEl9io,2022-05-08T21:56:49.000Z,"{'Date': '2022-05-08', 'Habit': ['rec24NGEkbvk..."
3,2022-05-07,[rec24NGEkbvktkD9B],3,[recenm1UYUwAIcoya],recqlcZ0qzaBZ7EHW,2022-05-08T17:11:28.000Z,"{'Date': '2022-05-07', 'Habit': ['rec24NGEkbvk..."


In [45]:
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

In [46]:
dfr

,Score
Date,
2022-05-07,8
2022-05-08,9


In [31]:
goal_table=Table(api_key, base_id, 'Goals')


In [35]:
goal_table.get('recenm1UYUwAIcoya')['fields']['Goal']

'Personal Development'